In [4]:
import numpy as np
import cupy as cp
# import cupy as np
import pandas as pd
from collections import OrderedDict
import re
import json
from tqdm import tqdm
import pickle
import sys
np.set_printoptions(threshold=sys.maxsize)

In [5]:
def exit_op():
    with open('./RNN_model.pkl','wb') as f:
        pickle.dump(model, f)

In [6]:
class Skip_Gram:
    def __init__(self,word_count,window = 4,hidden_units = 300):
        self.window = window
        self.learningrate = 1e-12
        len_word = len(word_count)
        self.word_count = word_count
        self.hidden_units = hidden_units
        self.len_word = len_word
        self.reseter()
        self.Weight = {}
        self.Weight["W"+str(1)] = np.random.normal(-1., 1.,(len_word,hidden_units)) 
        self.Weight["W"+str(2)] = np.random.normal(-1., 1.,(hidden_units,len_word))
        # self.Weight["W"+str(1)] = np.random.randn(len_word,hidden_units)/2.5+1
        # self.Weight["W"+str(2)] = np.random.randn(len_word,hidden_units)
        # self.Weight["W"+str(2)] = np.random.randn(hidden_units,len_word)/2.5+1
    def reseter(self):
        self.onehoty = cp.zeros((self.len_word,1))
        self.onehotx = cp.zeros((self.len_word,1))
    def softmax(self, x):
        max_val = np.max(x)
        x = x - max_val
        # if np.isnan(x).any():
        #     print("NAN")
        #     print("\n",max_val,"\n")
        #     exit_op()
        #     exit()
        x = np.exp(x)
        self.soft = x/np.sum(x)
        return self.soft
    def linear_ohot(self,x,x1):
        self.iidx = self.word_count[re.sub(r'[^A-Za-z0-9\\]+','', x ).lower()][1]
        self.onehotx[self.iidx] +=1
        y = np.dot(self.onehotx.T,x1)
        return y
    def linear(self,x,x1):
        self.x_out = x
        y = np.dot(x,x1)
        return y
    def loss(self,y):    
        hadamard = np.multiply(np.log(y+1e-25),self.onehoty.T).T
        doter = np.dot(self.onehoty.T,hadamard)
        loss = -np.sum(doter)
        return loss
    def forward(self,x,y):
        x = self.linear_ohot(x,self.Weight['W1'])
        for idx,target in enumerate(y):
            self.onehoty[self.word_count[re.sub(r'[^A-Za-z0-9\\]+','', target ).lower()][1]] += 1 
        x = self.softmax(self.linear(x,self.Weight['W2']))
        y = self.loss(x)
        return y
    def backward(self,x):
        x =  self.soft - self.onehoty.T
        b = np.multiply(self.onehoty.T,x)
        dwo = np.dot(self.x_out.T,b)
        self.Weight['W2'] -= self.learningrate*dwo
        y = np.dot(self.Weight['W2'],x.T)
        dwi = np.multiply(self.onehotx.reshape(self.onehotx.shape[0],1),y.T)
        # if np.isnan(dwi).any():
        #     print(self.onehotx)
        self.Weight['W1'] -= self.learningrate*dwi
        self.reseter()
    def predict(self,x):
        x = self.linear_ohot(x,self.Weight['W1'])
        x = self.softmax(self.linear(x,self.Weight['W'+str(2)]))
        self.reseter()
        return x
    def ss(self,x):
        text = x
        x = self.linear_ohot(x,self.Weight['W1'])
        x_values = x.reshape(300)
        x = self.linear(x,self.Weight['W'+str(2)])
        y_values = x
        self.reseter()
        word_wieght.append(x_values.get())
    def wv(self,x):
        self.reseter()
        x = self.linear_ohot(x,self.Weight['W1'])
        return x.reshape(1,self.hidden_units)
dataset = pd.read_csv("train1.csv")
with open("word_count.json",'r') as f:
    word_count = OrderedDict(json.load(f))
with open("w2v_model.pkl","rb") as f:
    model1 = pickle.load(f)

In [7]:
class relu:
    def __init__(self):
        self.mask = None
        self.x_shape = None
    def forward(self,x):
        self.x_shape = x.shape
        self.mask = x<=0
        x[self.mask] = 0
        y = x.reshape(self.x_shape)
        return y
    def backward(self,x):
        x[self.mask] = 0
        return x

In [8]:
class tanh:
    def __init__(self):
        self.tanh_value = None
        self.x = None
    def forward(self,x):
        self.x = x
        self.tanh_value = np.tanh(x)
        return self.tanh_value
    def backward(self):
        return 1 - self.tanh_value*self.tanh_value
    def tan_value(self):
        return self.tanh_value
    def x_value(self):
        return self.x 

In [9]:
class linear:
    def __init__(self):
        self.Wi = None
        self.Wh = None
        self.x = None
        self.htl = None
    def forward(self,x,wi,htl,wh,b):
        self.Wi = wi
        self.Wh = wh
        self.x = x
        self.htl = htl
        x1 = np.dot(wi,x.T)
        x2 = np.dot(wh,htl)
        y = np.add(x1,x2)
        y = np.add(y,b)
        return y
    def value_wi(self):
        return self.Wi
    def value_wh(self):
        return self.Wh
    def value_x(self):
        return self.x
    def value_htl(self):
        return self.htl

In [10]:
class out_layer:
    def __init__(self):
        self.soft = None
        self.t = None
        self.y = None
    def loss(self,y,t):    
        self.t = t
        self.y = y
        doter = np.dot(t.T,np.log(y+1e-8))
        loss = -np.sum(doter)
        return loss    
    def softmax(self,x):
        max_val = np.max(x)
        x = x - max_val
        x = np.exp(x)
        self.soft = x/np.sum(x)
        return self.soft
    def predict(self,x):
        x = self.softmax(x)
        x = x.argmax()
        return x
    def forward(self,x,t):
        x = self.softmax(x)
        y = self.loss(x,t)
        return y
    def backward(self):
        y = self.y - self.t
        return y

In [11]:
class RNN:
    def __init__(self,word_count,hidden_size=300):
        self.word_count = word_count
        self.word_len = len(word_count)
        self.voc = list(word_count.keys())
        self.hidden_size = hidden_size
        self.lr = 1e-1
        self.Weight = {}
        self.Weight["Wh"] = np.random.normal(0, np.sqrt(2.0/(hidden_size)),(hidden_size,hidden_size))
        self.Weight["Wo"] = np.random.normal(0, np.sqrt(2.0/(hidden_size)),(self.word_len,hidden_size))
        self.Weight["Wi"] = np.random.normal(0, np.sqrt(2.0/(hidden_size)),(hidden_size,hidden_size))
        self.Weight["bh"] = np.zeros((hidden_size,1))
        self.Weight["bo"] = np.zeros((self.word_len,1))
        self.h0 = np.zeros((self.hidden_size,1))
    def reseter(self):
        self.onehoty = np.zeros((self.word_len,1))
        self.onehotx = np.zeros((self.word_len,1))
    def linear(self,x,w,b):
        y = np.dot(w,x)
        y += b
        return y 
    def lookup(self,x):
        return self.word_count[re.sub(r'[^A-Za-z0-9\\]+','', x ).lower()][1]
    def forward(self,words):
        self.loss_value = 0
        self.layer = []
        htl = self.h0
        len_words = len(words)-1
        self.len_words = len_words
        for idx,word in enumerate(words):
            self.reseter()
            self.layer_h = []
            self.onehotx[self.lookup(word)] += 1
            embedd = model1.wv(word).get()
            self.layer_h.append(linear())
            x = self.layer_h[-1].forward(embedd,self.Weight["Wi"],htl,self.Weight['Wh'],self.Weight['bh'])
            self.layer_h.append(tanh())
            x = self.layer_h[-1].forward(x)
            htl = x
            x = self.linear(x,self.Weight['Wo'],self.Weight['bo'])
            self.layer_h.append(relu())
            x = self.layer_h[-1].forward(x)
            self.layer_h.append(out_layer())
            if idx < len_words:
                self.onehoty[self.lookup(words[idx+1])] += 1
            else:
                self.onehoty[self.lookup("\\e")] += 1
            y = self.layer_h[-1].forward(x,self.onehoty)
            self.layer.append(self.layer_h)
            self.loss_value += y
        return self.loss_value/len_words
    def backward(self):
        self.layer.reverse()
        htl = np.zeros_like(self.h0)
        wi = np.zeros_like(self.Weight['Wi'])
        wh = np.zeros_like(self.Weight['Wh'])
        wo = np.zeros_like(self.Weight['Wo'])
        bo = np.zeros_like(self.Weight['bo'])
        bh = np.zeros_like(self.Weight['bh'])
        for layer in self.layer:
            dy = layer[3].backward()
            dy = layer[2].backward(dy)
            bo += dy
            wo += np.dot(dy,layer[1].tan_value().T)
            do = np.dot(self.Weight['Wo'].T,dy) 
            do = do + htl
            dh = layer[1].backward()*do
            bh += dh
            wh += np.dot(dh,layer[0].value_htl().T)
            wi += np.dot(dh,layer[0].value_x())
            htl = np.dot(self.Weight['Wh'].T,dh)
        self.Weight['Wi'] -= self.lr * wi 
        self.Weight['Wh'] -= self.lr * wh
        self.Weight['Wo'] -= self.lr * wo  
        self.Weight['bo'] -= self.lr * bo  
        self.Weight['bh'] -= self.lr * bh
    def language(self,word):
        str = ' '
        str_list = []
        htl = np.zeros_like(self.h0)
        for _ in range(6):
            self.reseter()
            self.layer_h = []
            self.onehotx[self.lookup(word)] += 1
            embedd = model1.wv(word).get()
            self.layer_h.append(linear())
            x = self.layer_h[-1].forward(embedd,self.Weight["Wi"],htl,self.Weight['Wh'],self.Weight['bh'])
            self.layer_h.append(tanh())
            x = self.layer_h[-1].forward(x)
            htl = x
            x = self.linear(x,self.Weight['Wo'],self.Weight['bo'])
            self.layer_h.append(relu())
            x = self.layer_h[-1].forward(x)
            self.layer_h.append(out_layer())
            word = self.layer_h[-1].predict(x)
            word = self.voc[word]
            str_list.append(word)
        print(str_list)
        str = str.join(str_list)
        return str
model = RNN(word_count)

In [12]:
model.language("skier")

['syn', 'searching', 'stood', 'magnifying', 'fortification', 'military']


'syn searching stood magnifying fortification military'

In [13]:
for _ in tqdm(dataset.index[:1000]):
    sentence = dataset.iloc[0]['target'].split(' ')
    y = model.forward(sentence)
    if _ % 100 ==0:
        print(y)
    model.backward()
exit_op()

  0%|          | 0/1000 [00:00<?, ?it/s]

12.136289647544015


 10%|█         | 101/1000 [00:20<03:02,  4.93it/s]

5.0407103575918715


 20%|██        | 201/1000 [00:41<02:42,  4.92it/s]

...

 80%|████████  | 801/1000 [02:49<00:43,  4.57it/s]

2.4041184705880196


 90%|█████████ | 901/1000 [03:11<00:21,  4.59it/s]

2.40215299913339


100%|██████████| 1000/1000 [03:33<00:00,  4.68it/s]


In [14]:
model.language("skier")

['skis', 'down', 'the', '\\s', 'ice', 'claims']


'skis down the \\s ice claims'

In [117]:
with open("weight.txt","w") as f:
    f.write(str(model.Weight['Wh']))
    f.write(str(model.Weight['Wi']))
    f.write(str(model.Weight['Wo']))